In [88]:
import gensim
from gensim.models.word2vec import Word2Vec
import gensim.downloader as api

from rutermextract import TermExtractor

import nltk
nltk.download("stopwords")
#--------#

from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation

import wget
import zipfile

from os import listdir
from os.path import isfile, join

import re

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/pihanya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [89]:
#model_name = 'glove-wiki-gigaword-100'
#model_name = 'word2vec-ruscorpora-300'
model_url = 'http://vectors.nlpl.eu/repository/20/182.zip'

word2vec_postfixes = ['_PROPN', '_NOUN', '_ADJ', '_VERB']
russian_stopwords = stopwords.words("russian")

In [90]:
mystem = Mystem() 
term_extractor = TermExtractor()

In [91]:
#model = api.load(model_name)

# загрузка word2vec модели из сети
if not 'model.bin' in listdir('.'):
    m = wget.download(model_url)

model_file = model_url.split('/')[-1]
with zipfile.ZipFile(model_file, 'r') as archive:
    stream = archive.open('model.bin')
    model = gensim.models.KeyedVectors.load_word2vec_format(stream, binary=True)

In [92]:
questions = []
answers = []

dir_name = './feb'

files = [f for f in listdir(dir_name) if isfile(join(dir_name, f))]
for file_name in files:
    file = open(join(dir_name, file_name), 'r')
    lines = file.readlines()
    for i in range(len(lines)):
        lines[i] = lines[i][:-1]
    
    question = lines[0]
    answer = ' '.join(lines[1:])
                
    questions.append(question)
    answers.append(answer)

In [23]:
#### хорошие словосочетания - все слова есть в словаре word2vec
# плохие словосочетания - есть хотя бы одно слово не из словаря word2vec
# хорошее слово - слово из словаря word2vec
# плохое слово - слово не из словаря word2vec


# find_metrics( report_text, [idx, question] ): [ (idx, bp_metric, gp_metric, bw_metric, gw_metric, s_metric) ]
# 0. приводим все слова из факов к нормальной форме

#    extract_keys( text ): (bad_phrases, good_phrases, bad_word, good_words)
# 1. получаем из обращения важные слова и словосочетания с нормализированными словами
# 2. словосочетания закидываем в функцию классификации результата, на выходе получаются:
#    хорошие словосочетания, плохие словосочетания, хорошие слова, плохие слова

# 3. проходимся по всей базе вопросов-ответов и пробуем найти совпадение с текстом обращения:
#        по плохим словосочетаниям (метрика bp_metric)
#        по хорошим словосочетаниям (метрика gp_metric)
#        по плохим словам (метрика bw_metric)
#        по хорошим словам (метрика gw_metric)
#    в качестве значения метрики можно использовать количество символов или слов !(потом изучим)!
#    находя в факовом вопросе словосочетание из обращение, вытираем словосочетание
#      и все его слова из соответствующей категории
#    находя в факовом вопросе слово, вытираем его

# 4. для оставшихся плохих посчитаем метрику ненайденных плохих слов и словосочетаний !(длина или кол-во слов)!
#    Метрики: nfb_metric

#    find_synonymous( good_words ): 
# 5. если остались хорошие слова и/или словосочетания, занимаемся поиском метрики синонимичности
#    для этого все хорошие словосочетания дробим на слова и формируем один большой список хороших слов
#    и для каждого слова находим N самых близких слов и кладем в общий пул синонимов 


# 5.1 в вопросах факов найдем количество слов из пула синонимов. это количество будет метрикой синонимичности:
#     syn_metric

In [24]:
# является ли данный текст фразой или словом
def is_phrase(text):
    return ' ' in text

# сопоставить слову словарное слово из word2vec
# map_word2vec_word( word ): (is_success, word2vec_word or source_word)
def map_word2vec_word(word):
    temp_tokens = [word + x for x in word2vec_postfixes]
    for temp_token in temp_tokens:
        try:
            model.word_vec(temp_token)
        except Exception:
            pass
        else:
            return (True, temp_token)
        
    return (False, word)

In [25]:
# выделение важных слов и словосочетаний
# find_tokens( text ): array of phrases and words
def find_tokens(text):
    tokens = []
    for term in term_extractor(text):
        term_extractor(text)
        tokens.append(term.normalized)
        
    return tokens

# нормализация текста
# normalize_words( text ): normalized_text
def normalize_words(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords\
              and token != " " \
              and token.strip() not in punctuation]
    return ' '.join(tokens)

In [49]:
# найти хорошие/плохие словосочетания и слова
# extract_keys( text ): (bad_phrases, good_phrases, bad_words, good_words) - all sets
def extract_keys(text):
    bad_phrases, good_phrases = [], []
    bad_words, good_words = [], []
    
    text = text.replace('?', '.')
    text = text.replace('!', '.')
    sentences = text.split('.')
    tokens = set()
    for sentence in sentences:
        for token in find_tokens(sentence):
            tokens.add(normalize_words(token))
    
    word_tokens = [token for token in tokens if not is_phrase(token)]
    phrase_tokens = [token for token in tokens if is_phrase(token)]
    for phrase in phrase_tokens:
        for word in phrase.split(' '):
            word_tokens.append(word)
    
    word_tokens = [token for token in tokens if not is_phrase(token)]
    phrase_tokens = [token for token in tokens if is_phrase(token)]
    
    # распределение фраз по категориям
    for phrase in phrase_tokens:
        words = phrase.split(' ')
        for word in words:
            success, _ = map_word2vec_word(word)
            if not success:
                bad_phrases.append(phrase)
                break
        else:
            good_phrases.append(phrase)
    
    # распределение слов по категориям
    for word in word_tokens:
        success, _ = map_word2vec_word(word)
        if not success:
            bad_words.append(word)
        else:
            good_words.append(word)
    
    return (set(bad_phrases), set(good_phrases), set(bad_words), set(good_words))

In [46]:
# значение метрики - длина фразы (в словах)
def metric_value(token):
    if len(token) == 0:
        return 0
    return len(token.split(' '))   

# поиск значений различных метрик
# syn_coeff - минимальное значение похожести слов в word2vec для поиска синонимов
# find_metric( report_text, question, syn_coeff ): ( metrics, hits )
# metrics - массив из метрик. порядок метрик такой же как в шапке метода
# hits - тапл из четырех массивов: найденные совпадения плохих словосочетаний,
#        хороших словосочетаний, плохих слов, хороших слов
def find_metric( report_text, question, syn_coeff=0.5 ):
    question = normalize_words(question)
    
    bp_metric = 0.0 # плохие словосочетания
    gp_metric = 0.0 # хорошие словосочетания
    bw_metric = 0.0 # плохие слова
    gw_metric = 0.0 # хорошие слова
    syn_metric = 0.0 # метрика по синонимам
    nfb_metric = 0.0 # ненайденные плохие слова и словосочетания
    nfg_metric = 0.0 # ненайденные хорошие слова и словосочетания
    
    bad_phrases, good_phrases, bad_words, good_words = extract_keys(report_text)
    
    # не найденные слова
    nf_bad_words, nf_good_words = set(), set()
    
    # найденные совпадения
    bad_phrases_hits, good_phrases_hits = set(), set()
    bad_words_hits, good_words_hits = set(), set()
    
    # поиск значения метрики плохих словосочетаний
    for phrase in bad_phrases:
        metric = metric_value(phrase)
        if phrase in question:
            bad_phrases_hits.add(phrase)
            bp_metric += metric
            
            # больше не ищем слова из словосочетания
            for word in phrase.split(' '):
                if word in bad_words:
                    bad_words.remove(word)
                if word in good_words:
                    good_words.remove(word)
        else:
            # если не нашли, тогда помещаем все плохие слова из плохих словосочетаний в пул ненайденных плохих слов 
            words = phrase.split(' ')
            for word in words:
                is_good, _ = map_word2vec_word(word)
                if not is_good:
                    nf_bad_words.add(word)
            
    # поиск значения метрики хороших словосочетаний
    for phrase in good_phrases:
        if phrase in question:
            good_phrases_hits.add(phrase)
            bp_metric += metric_value(phrase)
            
            # больше не ищем слова из словосочетания
            for word in phrase.split(' '):
                if word in good_words:
                    good_words.remove(word)
        else:
            # если не нашли, тогда помещаем все хорошие слова из словосочетания в пул ненайденных хороших слов 
            words = phrase.split(' ')
            for word in words:
                nf_good_words.add(word)
                
    
    # поиск значения метрики плохих слов
    for word in bad_words:
        if word in question:
            bad_words_hits.add(word)
            bw_metric += metric_value(word) # можно сделать просто инкремент
        else:
            nf_bad_words.add(word)
            
    # метрика ненайденных плохих слов
    nfb_metric += sum([metric_value(word) for word in nf_bad_words])
            
    # поиск значения метрики хороших слов
    for word in good_words:
        if word in question:
            good_words_hits.add(word)
            gw_metric += metric_value(word) # можно сделать просто инкремент
        else:
            nf_good_words.add(word)
    
    # поиск метрики синонимов
    for word in nf_good_words:
        _, w2v_word = map_word2vec_word(word)
        
        similiarities = model.similar_by_word(w2v_word)
        for similiarity in similiarities:
            syn_word, prob = similiarity
            if prob < syn_coeff:
                break
            
            metric = metric_value(syn_word)
            if syn_word in question:
                syn_metric += metric
            else:
                nfg_metric += metric
            
    metrics = [bp_metric, gp_metric, bw_metric, gw_metric, syn_metric, nfb_metric, nfg_metric]
    hits = (bad_phrases_hits, good_phrases_hits, bad_words_hits, good_words_hits)
    
    return (metrics, hits)


def print_metric(metrics, hits=None):
    titles = [
        'плохие словосочетания', 'хорошие словосочетания',
        'плохие слова', 'хорошие слова',
        'метрика по синонимам',
        'ненайденные плохие слова/сочетания', 'ненайденные хорошие слова/сочетания'
    ]
    for i in range(len(metrics)):
        print(titles[i], ':', metrics[i])

    if hits is not None:
        bp_hits, gp_hits, bw_hits, gw_hits = hits
        print('---')
        print('плохие словосочетания', ':', bp_hits)
        print('хорошие словосочетания', ':', gp_hits)
        print('плохие слова', ':', bw_hits)
        print('хорошие слова', ':', gw_hits)

In [55]:
# вопрос, который задал юзер сейчас
actual_question = 'Здравствуйте, можно ли изменить цену в оферте, которой уже присвоен номер? Или нужно создавать новую оферту?'
print('actual question:', actual_question)

# проходимся по всей базе вопросов
for i in range(min(10, len(questions))):
#for question in questions:
    question = questions[i]
    metric, hits = find_metric(actual_question, question)
    
    # смотрим на метрики, если сумма положительных метрик больше единицы, то печатаем вопрос из базы
    if sum(metric[:4]) > 1:
        print(question)
        print_metric(metric, hits)
        print('======')

actual question: Здравствуйте, можно ли изменить цену в оферте, которой уже присвоен номер? Или нужно создавать новую оферту?
не присваивается номер оферте, что делать?
плохие словосочетания : 0.0
хорошие словосочетания : 0.0
плохие слова : 1.0
хорошие слова : 1.0
метрика по синонимам : 0.0
ненайденные плохие слова/сочетания : 1.0
ненайденные хорошие слова/сочетания : 2.0
---
плохие словосочетания : set()
хорошие словосочетания : set()
плохие слова : {'оферта'}
хорошие слова : {'номер'}
Здравствуйте! Я зарегистрирован на портале как поставщик. Отправил заявку на создание оферты с внесением нового вида услуги в СТЕ через экранную форму. Сейчас открыл эту оферту. Номер оферты не присвоен. Статус: ввод сведений. Нет связи с СТЕ.Еще некоторые характеристики услуги пропали. Подскажите, это так и должно быть? Мне нужно подождать какое-то время для проверки и утверждения? Или я что-то неправильно сделал (не доделал)?
плохие словосочетания : 0.0
хорошие словосочетания : 0.0
плохие слова : 1.0


In [54]:
# вопрос, который задал юзер сейчас
actual_question = 'Здравствуйте, можно ли изменить цену в оферте, которой уже присвоен номер? Или нужно создавать новую оферту?'
question = 'не присваивается номер оферте, что делать?'
print('actual question:', actual_question)

print(extract_keys(question))
print(extract_keys('Здравствуйте, можно ли изменить цену в оферте, которой уже присвоен номер'))
print(extract_keys('Или нужно создавать новую оферту?'))
metric, hits = find_metric(actual_question, question)
    
# смотрим на метрики, если сумма положительных метрик больше единицы, то печатаем вопрос из базы
if sum(metric[:4]) > 1:
    print(question)
    print_metric(metric, hits)
    print('======')

actual question: Здравствуйте, можно ли изменить цену в оферте, которой уже присвоен номер? Или нужно создавать новую оферту?
(set(), set(), {'оферта'}, {'номер'})
(set(), set(), {'оферта'}, {'номер', 'цена'})
({'новый оферта'}, set(), set(), set())
не присваивается номер оферте, что делать?
плохие словосочетания : 0.0
хорошие словосочетания : 0.0
плохие слова : 1.0
хорошие слова : 1.0
метрика по синонимам : 0.0
ненайденные плохие слова/сочетания : 1.0
ненайденные хорошие слова/сочетания : 2.0
---
плохие словосочетания : set()
хорошие словосочетания : set()
плохие слова : {'оферта'}
хорошие слова : {'номер'}


In [61]:
# вопрос, который задал юзер сейчас
actual_question = 'В нашей заявке 2886444-19 мы ждем номер заявки на СТЕ. Как скоро обычно такие заявки рассматриваются?'
print('actual question:', actual_question)

# проходимся по всей базе вопросов
max_similarity = 0
found_index = None
idx = -1
for i in range(min(30, len(questions))):
#for question in questions:
    question = questions[i]
    print("NOW", question)
    metric, hits = find_metric(actual_question, question)
    print()
    if sum(metric[:5]) > max_similarity:
        max_similarity = sum(metric[:5])
        idx = i
        found_index = metric, hits
       
if max_similarity > 0:
    print(max_similarity, questions[idx])
    print_metric(metric, hits)
        
    
    # смотрим на метрики, если сумма положительных метрик больше единицы, то печатаем вопрос из базы
#     if sum(metric[:4]) > 1:
#         print(question)
#         print_metric(metric, hits)
#         print('======')

actual question: В нашей заявке 2886444-19 мы ждем номер заявки на СТЕ. Как скоро обычно такие заявки рассматриваются?
NOW Добрый день. Чтобы зарегистрировать оферту с товаром, которого нет в СТЕ (что-то новое на сайте), нужно подать заявку на СТЕ, а что дальше? Ждать?
NOW Здравствуйте! У Вас можно заказать эцп для портала на ИП
NOW не присваивается номер оферте, что делать?
NOW Добрый день. Зарегистрировали ЭЦП как ИП, но при подаче заявления форма для Юридического лица, как это исправить?
NOW При оформлении заявки на оказания услуг в разделе Регион отсутствует город Москва! Всего указано три региона! В ручную набрать регион невозможно. Отправить заявку невозможно! Что за беспредел?
NOW не работает!!!!!!!!!!!! есть кто? активна кнопка "копировать" до этого была ошибка, что неи индекса у фирмы заявку отправили на внесение изменений сколько ждать? где увидеть, что есть заявка на исправление? Статус: Ввод сведений нужно срочне зарегистрировать до обновы все работало! ИНН 7721305971
NOW Т

NOW Как правильно внести единицы измерения в количественные характеристики?
NOW здравствуйте, прохожу регистрацию, нажимаю отправить заявку, а она не отправляется
NOW Здравствуйте!Временный пароль выслан на Ваш электронный почтовый ящик "5283***mail.ru". К нам ничего не пришло. 5 раз просили послать повторно, тоже ничего не приходит
NOW Добрый вечер! Что писать в строчке : банковский идентификационный код? Спасибо разобрались, спасибо Другой вопрос: Заявку на регистрацию сделали, она в состоянии "Черновик" и не отправляется или так должно быть, когда рассмотрят измениться состояние?
NOW Добрый день! Необходимо сделать оферту на товар. Как ее сделать!? Проще то нельзя было все придумать то!? Помогите пожалуйста!
NOW Добрый день. Почему не отравляется заявка на регистрацию?
NOW Добрый день! Белокобыльский Виталий Николаевич При подписании оферты выдает ошибку Для продолжения подписания оферты необходимо устранить ошибки в профиле компании: Поле "Индекс (фактический)" обязательно для запо

In [98]:
questions

['Добрый день. Чтобы зарегистрировать оферту с товаром, которого нет в СТЕ (что-то новое на сайте), нужно подать заявку на СТЕ, а что дальше? Ждать?',
 'Здравствуйте! У Вас можно заказать эцп для портала на ИП',
 'не присваивается номер оферте, что делать?',
 'Добрый день. Зарегистрировали ЭЦП как ИП, но при подаче заявления форма для Юридического лица, как это исправить?',
 'При оформлении заявки на оказания услуг в разделе Регион отсутствует город Москва! Всего указано три региона! В ручную набрать регион невозможно. Отправить заявку невозможно! Что за беспредел?',
 'не работает!!!!!!!!!!!! есть кто? активна кнопка "копировать" до этого была ошибка, что неи индекса у фирмы заявку отправили на внесение изменений сколько ждать? где увидеть, что есть заявка на исправление? Статус: Ввод сведений нужно срочне зарегистрировать до обновы все работало! ИНН 7721305971',
 'Тестовое обращение.',
 'добрый день. продлила свой ключ еще на год.как мне его прописать на сайте?',
 'Здравствуйте! Я зар

In [96]:
# вопрос, который задал юзер сейчас
def get_simple_answer(request):
    # проходимся по всей базе вопросов
    max_similarity = 0
    found_index = None
    idx = -1
    for i in range(min(30, len(questions))):
        question = questions[i]
        print("NOW", question)
        metric, hits = find_metric(request, question)
        print(metric[:5])
        if sum(metric[:5]) > max_similarity:
            max_similarity = sum(metric[:5])
            idx = i
            found_index = metric, hits

    if max_similarity > 0:
        print(max_similarity, questions[idx])
        print("ANSWER", answers[idx])
        print_metric(metric, hits)
        
    text = request.replace('?', '.')
    text = text.replace('!', '.')
    sentences = text.split('.')
    tokens = set()
    print(term_extractor(request))
    for sentence in sentences:
        for token in find_tokens(sentence):
            tokens.add(token)
            
    for token in tokens:
        request = request.replace(token, token.upper())
    return answers[idx], request, tokens
            
            
actual_question = 'В нашей заявке 2886444-19 мы ждем номер заявки на СТЕ. Как скоро обычно такие заявки рассматриваются?'
get_simple_answer(str(actual_question))

    
        
    
    # смотрим на метрики, если сумма положительных метрик больше единицы, то печатаем вопрос из базы
#     if sum(metric[:4]) > 1:
#         print(question)
#         print_metric(metric, hits)
#         print('======')

NOW Добрый день. Чтобы зарегистрировать оферту с товаром, которого нет в СТЕ (что-то новое на сайте), нужно подать заявку на СТЕ, а что дальше? Ждать?
[0.0, 0.0, 0.0, 1.0, 0.0]
NOW Здравствуйте! У Вас можно заказать эцп для портала на ИП
[0.0, 0.0, 0.0, 0.0, 0.0]
NOW не присваивается номер оферте, что делать?
[0.0, 0.0, 0.0, 0.0, 0.0]
NOW Добрый день. Зарегистрировали ЭЦП как ИП, но при подаче заявления форма для Юридического лица, как это исправить?
[0.0, 0.0, 0.0, 0.0, 0.0]
NOW При оформлении заявки на оказания услуг в разделе Регион отсутствует город Москва! Всего указано три региона! В ручную набрать регион невозможно. Отправить заявку невозможно! Что за беспредел?
[0.0, 0.0, 0.0, 1.0, 0.0]
NOW не работает!!!!!!!!!!!! есть кто? активна кнопка "копировать" до этого была ошибка, что неи индекса у фирмы заявку отправили на внесение изменений сколько ждать? где увидеть, что есть заявка на исправление? Статус: Ввод сведений нужно срочне зарегистрировать до обновы все работало! ИНН 77213

('Здравствуйте .Регламент рассмотрения заявки на СТЕ- 5 дне',
 'В нашей заявке 2886444-19 мы ждем НОМЕР ЗАЯВКИ на СТЕ. Как скоро обычно такие заявки рассматриваются?',
 {'наша заявка', 'номер заявки', 'такие заявка'})